In [2]:
import sys, os, re, csv, codecs 
import numpy as np
import pandas as pd, configparser

# gensim
from gensim.models import FastText
from gensim.test.utils import get_tmpfile

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model

# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [3]:
config = configparser.ConfigParser()
config.read('../config.cfg')
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [4]:
# Chargement des données sous forme de dataframes pandas
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [5]:
# pour créer un modèle Word2vec, il faut transformer le texte en liste de mots (tokens). 
# On retire les stopWords et la ponctuaction pour essayer de coller au modèle Google
sentences = []
stopWords = set(stopwords.words('english'))
all_comment = list_sentences_train.tolist() + list_sentences_test.tolist()
for comment in all_comment:
    comment = ' '.join(re.split('\n',comment.lower()))
    comment = re.sub(r'[^a-zA-Z0-9\s:]', '', comment)
    words = word_tokenize(comment)
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    sentences.append(wordsFiltered)

In [13]:
model = FastText(size=300, sentences=sentences, iter=10)

In [14]:
fname = get_tmpfile("fasttext.model")
model.save(fname)
model = FastText.load(fname)

https://radimrehurek.com/gensim/models/fasttext.html
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/FastText_Tutorial.ipynb


In [15]:
word_vectors = model.wv

In [9]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

In [10]:
# A des fins de comparaison, on utilise le même tokenizer que pour le notebook Google

# On garde les mots les plus fréquents dans le jeu d'entrainement pour établir notre liste de tokens.
tokenizer = Tokenizer(num_words=max_features)
# Calcul des mots les plus fréquents
tokenizer.fit_on_texts(list(list_sentences_train))
# Indexation des jeux d'entrainement et test, conversion du texte en séquence d'indexes
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [11]:
# Les commentaires ont une longueur variable qui peut être inférieure à maxlen
# On complète donc la séquence avec des 0, par défaut au début de la séquence
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [16]:
word_index = tokenizer.word_index
# le modèle pré-entrainé contient 3 millions de mots contre 20000 pour notre jeu de données.
vocabulary_size=min(len(word_index)+1,max_features)

# la matrice d'embedding représente le vocabulaire final à utiliser. 
# On ne gardera que 20000 vecteurs du modèle pré-entrainé. Leur longueur est de 300.
embedding_matrix = np.zeros((vocabulary_size, embed_size))


for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector    
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

In [19]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 734s 5ms/step - loss: 0.0553 - acc: 0.9806 - val_loss: 0.0477 - val_acc: 0.9821
Epoch 2/2
143613/143613 [==============================] - 715s 5ms/step - loss: 0.0447 - acc: 0.9831 - val_loss: 0.0460 - val_acc: 0.9828


In [21]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [22]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 48s 756us/step


[0.06498602694861513, 0.9732695033466862]

In [23]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [24]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

0.8869924036387509


In [25]:
# Classification pour chaque label de façon indépendante
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )

... Traitement du label toxic
L'accuracy du jeu test est 0.929835255869205
Matrice de confusion sur le jeu test:
[[54620  3268]
 [ 1221  4869]]
Rapport : 
              precision    recall  f1-score   support

           0       0.98      0.94      0.96     57888
           1       0.60      0.80      0.68      6090

   micro avg       0.93      0.93      0.93     63978
   macro avg       0.79      0.87      0.82     63978
weighted avg       0.94      0.93      0.93     63978

... Traitement du label severe_toxic
L'accuracy du jeu test est 0.994248022757823
Matrice de confusion sur le jeu test:
[[63570    41]
 [  327    40]]
Rapport : 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     63611
           1       0.49      0.11      0.18       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.74      0.55      0.59     63978
weighted avg       0.99      0.99      0.99     63978

... Traitement du label obsce